In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import time
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
!pip install feather-format >> none
!pip install faiss-cpu --no-cache
!pip install implicit
import implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 18.1 MB/s eta 0:00:00


I get non-repetitive url_hosts

In [4]:
urls = pd.read_feather('/data preprocessing/data/dataset_full.feather')
urls = urls['url_host'].drop_duplicates()
urls

0                                ad.adriver.ru
1                                    apple.com
2                       avatars.mds.yandex.net
3                  googleads.g.doubleclick.net
5                                  i.ytimg.com
                           ...                
322861818                        poliglot1.com
322872650                     reservemaster.ru
322882531                       uploadhive.com
322883336            kuzma2012.livejournal.com
322888794    stroitelnye-materialy-v-kurske.ru
Name: url_host, Length: 199683, dtype: category
Categories (199683, object): ['-1', '0--stranger-livejournal-com.turbopages.org', '0-1.ru', '0-34.ru', ..., 'юбилей-на-бис.рф', 'южныйокруг.рф', 'явернусь.рф', 'яркнига24.рф']

Function for writing sites data to csv-file

In [5]:
import csv
def csv_writer(data):
    with open('/data/sites.csv', 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(data)

### Function for reading information from sites and writing it to file
In this function I read html-page by url, parse description, title and keywords from html and write it to csv-file. There are 5 urls that crash script and I had to remove them:
- sanstv.ru
- t.lordfilms-filmy.online
- z.lordfilms-film.online
- n.lordfilm-smotret.one
- z.lordfilmy-film.online

In [6]:
%%time
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
from urllib3.exceptions import LocationParseError

def parse_url(url):
    url_data = np.empty([5], dtype=object)
    url_data[0] = url[0]
    url_data[1] = url[1]
    try:
        if (url[1] == 'sanstv.ru'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 't.lordfilms-filmy.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'z.lordfilms-film.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'n.lordfilm-smotret.one'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'z.lordfilmy-film.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        html = requests.get(f"http://{url[1]}", timeout=6, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 
                                                           'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
                                                           'Accept-Encoding': 'gzip, deflate, br',
                                                           'Accept-Language':  'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
                                                           'Cookie': 'vid=1b448c0013086e08644a; mkmgsgp=RQXPUS'})
        # Due to problems with encoding in sites I forced decode text as utf-8 and I for bad decoded sites I use cp1251
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, 'lxml')
        descr = soup.find('meta', attrs={"name": "Description", 'content' : True})
        if descr is not None:
            url_data[3] = descr['content']
        descr = soup.find('meta', attrs={"name": "description", 'content' : True})
        if descr is not None:
            url_data[3] = descr['content']
        del(descr)
        title = soup.find('title')
        if title is not None:
            url_data[2] = title.text
        del(title)
        keywords = soup.find('meta', attrs={"name": "keywords", 'content' : True})
        if keywords is not None:
            url_data[4] = keywords['content']
        keywords = soup.find('meta', attrs={"name": "Keywords", 'content' : True})
        if keywords is not None:
            url_data[4] = keywords['content']
        del(keywords)
        csv_writer(url_data)
        del(html)
        del(soup)     
        del(url_data)
    except AttributeError: 
        csv_writer(url_data)       
        del(url_data)
    except requests.exceptions.RequestException: 
        csv_writer(url_data)       
        del(url_data)
    except LocationParseError: 
        csv_writer(url_data)       
        del(url_data)  


i = 0
j = 5000
while (j < 199684):
    # I use 40 thread for parsing
    with Pool(40) as p: 
        p.map(parse_url, enumerate(urls[urls.index[range(i, j)]]))
    i = j
    print(i)
    if (j != 195000):
        j += 5000
    else:
        j = 199683

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
199683
CPU times: user 32.8 s, sys: 59.7 s, total: 1min 32s
Wall time: 4h 3min 18s
